In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import torch as torch


In [ ]:
# Add the project root path if not already present
PROJECT_ROOT = os.path.abspath("..")  # move up one level from notebooks/
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Visual confirmation
print("[✓] Project at:", PROJECT_ROOT)

from src.data.load_data import load_train_data, EDGCDataset
from src.data.stratified_split import stratified_split_pad_torch
from src.models.model_trainer import Trainer

[✓] Project at: c:\Users\toby_\Documents\TU_Berlin\Semestre 3\AMLS\AMLS_packed


In [6]:
X_train, y_train = load_train_data()

durations = np.array([len(x) / 300 for x in X_train])

cls_count = y_train[0].groupby(y_train[0]).count()

[✓] Loaded X_train with 6179 sequences
[✓] Loaded y_train with shape (6179, 1)


In [7]:
X_train, X_val, lengths_train, lengths_val, y_train, y_val = stratified_split_pad_torch(
    X_train, y_train
)

print(X_train.shape, y_train.shape)




train_dataset = EDGCDataset(X_train, lengths_train, y_train)
val_dataset = EDGCDataset(X_val, lengths_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

torch.Size([5564, 18260]) torch.Size([5564, 1])


# Model Definition

## ECGNet Model Architecture (CNN-LSTM model)

The `ECGNet` class implements a neural network for ECG signal classification. The architecture consists of:

- **Spectrogram transformation**: Converts raw ECG signals into spectrograms using Short-Time Fourier Transform (STFT).
- **Convolutional layers**: Two sequential 2D convolutional layers with ReLU activations and max pooling, which extract spatial features from the spectrogram.
- **Recurrent layer**: An LSTM layer processes the sequence of features, capturing temporal dependencies.
- **Fully connected layer**: The final linear layer maps the LSTM output to the target number of classes.

The model expects ECG signals and their lengths as input, and produces class logits for classification tasks.

The `ECGNet` class is implemented on the module /models/model_1.py  

In [8]:
from src.models.model_1 import ECGNet
from src.models.hyperparamter_tunning import hyperparameter_search

In [9]:
print(f"X_train: {X_train.shape}")
print(f"lengths_train: {lengths_train.shape}")
print(f"y_train: {y_train.shape}")

X_train: torch.Size([5564, 18260])
lengths_train: torch.Size([5564])
y_train: torch.Size([5564, 1])


### 🔍 Hyperparameter Search and Model Selection

In this section, we perform **hyperparameter tuning** to improve the performance of our machine learning models.

When training neural networks, model performance is highly sensitive to hyperparameters such as the number of layers, hidden units, learning rate, dropout rate, and more. Manually selecting these values is inefficient and often suboptimal. Therefore, we use a **grid search strategy** to systematically explore combinations of hyperparameter values.

---

#### ✅ Why Hyperparameter Tuning?

- Different combinations can lead to **very different results**, even with the same architecture.
- Some configurations may **overfit** or **underfit**, while others may **generalize better**.
- Automated tuning helps us identify the **best performing model** on the validation set without manual trial-and-error.

---

#### ⚙️ How it Works

1. We define a **search space**, i.e., a dictionary of hyperparameter values to test.
2. We generate all **combinations** using Cartesian product (`itertools.product`).
3. For each combination:
   - Initialize the model with the current hyperparameters.
   - Train it for a fixed number of epochs on the training set.
   - Evaluate its performance on the validation set using **F1-score** and **loss**.
4. Store and compare results, selecting the configuration that performs **best on validation data**.

This process is repeated for each of the two model architectures used in this project:

- A **CNN-LSTM hybrid** based on time-frequency STFT features.
- A **Temporal Convolutional Network (TCN)** using a stacked 1D convolutional architecture.

---

By applying this strategy, we ensure our final models are both **well-tuned** and **generalizable**, which is essential for real-world performance.


In [12]:
# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_grid = {
    "lst_hidden_size": [32, 64, 128],
    
    "learning_rate": [.01, 0.001, 0.0005],
    
    "dropout": [0.1, 0.2, .5],
}
fixed = {
    "num_classes": 4,
    "signal_length": X_train.shape[1],
    "n_fft": 512,
    "hop_length": 256,
    "conv1_padding": 1,
    "conv2_padding": 1,
    "conv1_kernel": 3,
    "conv2_kernel": 3,
    "lstm_num_layers": 1,
    "conv1_channels": 32,
    "conv2_channels": 32
}

results = hyperparameter_search(
    ECGNet,
    param_grid,
    fixed,
    device=device,
    epochs=7,
    train_loader=train_loader,
    val_loader=val_loader,
)


🔧 Training with config: {'num_classes': 4, 'signal_length': 18286, 'n_fft': 512, 'hop_length': 256, 'conv1_padding': 1, 'conv2_padding': 1, 'conv1_kernel': 3, 'conv2_kernel': 3, 'lstm_num_layers': 1, 'conv1_channels': 32, 'conv2_channels': 32, 'lst_hidden_size': 32, 'learning_rate': 0.01, 'dropout': 0.1}
Epoch 1/7 | Train Loss: 1.0238 | Train F1: 0.1853 | Val Loss: 1.0021 | Val F1: 0.1856
Epoch 2/7 | Train Loss: 1.0173 | Train F1: 0.1853 | Val Loss: 1.0039 | Val F1: 0.1856
Epoch 3/7 | Train Loss: 1.0159 | Train F1: 0.1853 | Val Loss: 1.0058 | Val F1: 0.1856
Epoch 4/7 | Train Loss: 1.0139 | Train F1: 0.1863 | Val Loss: 1.0019 | Val F1: 0.1856
Epoch 5/7 | Train Loss: 1.0156 | Train F1: 0.1853 | Val Loss: 1.0137 | Val F1: 0.1856
Epoch 6/7 | Train Loss: 1.0143 | Train F1: 0.1856 | Val Loss: 0.9956 | Val F1: 0.1856
Epoch 7/7 | Train Loss: 1.0069 | Train F1: 0.1853 | Val Loss: 0.9901 | Val F1: 0.1856

🔧 Training with config: {'num_classes': 4, 'signal_length': 18286, 'n_fft': 512, 'hop_leng

### 🧪 Best Hyperparameter Configuration for CNN-LSTM model

After performing an extensive grid search over several key hyperparameters of the `CNN-LSTM` architecture, the best-performing configuration (based on validation F1-score) was identified as:

- `dropout = .1`
- `lst_hidden_size = 128`
- `learning_rate = 0.001`

This combination yielded the highest balance between training and validation performance, indicating that the model can generalize well without overfitting.

---

### 🚀 Final Training Setup

With the optimal hyperparameters selected, we now proceed to **train the final version of CNN-LSTM** using a larger number of epochs (50) to fully exploit the model's capacity:


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ECGNet(
    num_classes=4,
    n_fft=512,
    hop_length=256,
    conv1_padding=1,
    conv2_padding=1,
    conv1_kernel=3,
    conv2_kernel=3,
    lstm_num_layers=1,
    conv1_channels=32,
    conv2_channels=32,
    lst_hidden_size=128,
    dropout=0.1,
    signal_length=X_train.shape[1],
    device=device,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = torch.nn.CrossEntropyLoss()

trainer = Trainer(model, optimizer, criterion, augment_data=False, device=device)

history = trainer.fit(train_loader, val_loader, epochs=50)

train_loss, train_f1 = trainer.evaluate(train_loader)

val_loss, val_f1 = trainer.evaluate(val_loader)

print(train_f1, val_f1)

Epoch 1/50 - Train Loss: 0.9960 - Train F1: 0.1986 - Val Loss: 0.9599 - Val F1: 0.1856
Epoch 2/50 - Train Loss: 0.9660 - Train F1: 0.2024 - Val Loss: 0.9607 - Val F1: 0.1856
Epoch 3/50 - Train Loss: 0.9492 - Train F1: 0.2190 - Val Loss: 0.8977 - Val F1: 0.2600
Epoch 4/50 - Train Loss: 0.8864 - Train F1: 0.2894 - Val Loss: 0.8050 - Val F1: 0.3191
Epoch 5/50 - Train Loss: 0.8137 - Train F1: 0.3508 - Val Loss: 0.7638 - Val F1: 0.3429
Epoch 6/50 - Train Loss: 0.7776 - Train F1: 0.4056 - Val Loss: 0.7392 - Val F1: 0.3650
Epoch 7/50 - Train Loss: 0.7293 - Train F1: 0.4760 - Val Loss: 0.6795 - Val F1: 0.5042
Epoch 8/50 - Train Loss: 0.6912 - Train F1: 0.5241 - Val Loss: 0.6486 - Val F1: 0.5731
Epoch 9/50 - Train Loss: 0.6588 - Train F1: 0.5856 - Val Loss: 0.6316 - Val F1: 0.5411
Epoch 10/50 - Train Loss: 0.6332 - Train F1: 0.6048 - Val Loss: 0.6133 - Val F1: 0.5747
Epoch 11/50 - Train Loss: 0.6066 - Train F1: 0.6560 - Val Loss: 0.6072 - Val F1: 0.6544
Epoch 12/50 - Train Loss: 0.5907 - Train 

### 🔍 Hyperparameter Search and Model Selection (TCN)

In this section, we perform **hyperparameter tuning** for the `TCN_STFT_Classifier` model, which combines a Temporal Convolutional Network (TCN). The goal is to identify the best-performing configuration of this architecture on the validation set.

---

#### ⚙️ How it Works

1. We define a **grid of hyperparameters** to test, including:
   - `dropout` rates
   - TCN `hidden_channels` per layer
   - `kernel size`
   - `num levels`
2. Using `itertools.product`, we generate all possible **combinations** from this grid.
3. For each combination:
   - Instantiate the model with the current hyperparameters.
   - Train it for a fixed number of epochs (e.g., 10) on the training set.
   - Evaluate its performance using **F1-score** and **loss** on the validation set.
4. Track the best configuration based on **validation F1-score**.

---

This tuning process ensures that our TCN model, augmented with time-frequency features from STFT, is both **optimized** and **robust** to overfitting or underfitting. It complements the tuning process performed on our first model (`ECGNet`), allowing us to compare both architectures fairly under their best conditions.


In [13]:
from src.models.model_2 import TCN_STFT_Classifier


param_grid = {
    # Configuraciones donde len(hidden_channels) == num_levels
    
        'hidden_channels': [[64,128,128,128],[128,128,128,128]],
    'dropout': [0.1, 0.2, 0.3],
    'kernel_size': [3, 5],
    'num_levels': [3,4]
}


fixed = {
    "num_classes": 4,
    "n_fft": 256,
    "hop_length": 128,
    "kernel_size": 3,
    "learning_rate" : .001,
}

results = hyperparameter_search(
    TCN_STFT_Classifier,
    param_grid,
    fixed,
    device=device,
    epochs=5,
    train_loader=train_loader,
    val_loader=val_loader,
)


🔧 Training with config: {'num_classes': 4, 'n_fft': 256, 'hop_length': 128, 'kernel_size': 3, 'hidden_channels': [64, 128, 128], 'dropout': 0.1, 'num_levels': 3}
Learning rate: 0.001


c:\Users\toby_\AMLS\venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/5 | Train Loss: 0.8957 | Train F1: 0.3041 | Val Loss: 0.7980 | Val F1: 0.4038
Epoch 2/5 | Train Loss: 0.7564 | Train F1: 0.4473 | Val Loss: 0.7375 | Val F1: 0.4446
Epoch 3/5 | Train Loss: 0.6787 | Train F1: 0.5305 | Val Loss: 0.6653 | Val F1: 0.6193
Epoch 4/5 | Train Loss: 0.6359 | Train F1: 0.5815 | Val Loss: 0.6691 | Val F1: 0.5076
Epoch 5/5 | Train Loss: 0.6122 | Train F1: 0.5919 | Val Loss: 0.6083 | Val F1: 0.5734

🔧 Training with config: {'num_classes': 4, 'n_fft': 256, 'hop_length': 128, 'kernel_size': 3, 'hidden_channels': [64, 128, 128, 128], 'dropout': 0.1, 'num_levels': 4}
Learning rate: 0.001


c:\Users\toby_\AMLS\venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/5 | Train Loss: 0.9070 | Train F1: 0.2869 | Val Loss: 0.8204 | Val F1: 0.3681
Epoch 2/5 | Train Loss: 0.7834 | Train F1: 0.4224 | Val Loss: 0.7378 | Val F1: 0.4814
Epoch 3/5 | Train Loss: 0.7238 | Train F1: 0.4984 | Val Loss: 0.6812 | Val F1: 0.5096
Epoch 4/5 | Train Loss: 0.6698 | Train F1: 0.5475 | Val Loss: 0.6347 | Val F1: 0.4769
Epoch 5/5 | Train Loss: 0.6413 | Train F1: 0.5732 | Val Loss: 0.6350 | Val F1: 0.5744

🔧 Training with config: {'num_classes': 4, 'n_fft': 256, 'hop_length': 128, 'kernel_size': 5, 'hidden_channels': [64, 128, 128], 'dropout': 0.1, 'num_levels': 3}
Learning rate: 0.001
Epoch 1/5 | Train Loss: 0.9172 | Train F1: 0.2912 | Val Loss: 0.8757 | Val F1: 0.3684
Epoch 2/5 | Train Loss: 0.8034 | Train F1: 0.4009 | Val Loss: 0.7960 | Val F1: 0.4224
Epoch 3/5 | Train Loss: 0.7169 | Train F1: 0.4890 | Val Loss: 0.6728 | Val F1: 0.5141
Epoch 4/5 | Train Loss: 0.6523 | Train F1: 0.5715 | Val Loss: 0.6141 | Val F1: 0.6033
Epoch 5/5 | Train Loss: 0.6229 | Train F1:

### 🏁 Final Model Selection: (TCN)

After conducting an extensive grid search over multiple combinations of hyperparameters, we identified the **best-performing configuration** for the `(TCN)` model.

---

#### 🔧 Best Hyperparameters Found

- `kernel size = 3`
- `num levels = 3`
- `dropout = 128`
- `hidden channels = [64, 128, 128]`

These hyperparameters achieved the **highest F1-score** on the validation set among all tested configurations.

---

#### 🚀 Final Training Phase

Using this optimal setup, we now train the final version of the `TCN` model using **50 epochs** to allow the model to fully converge and leverage the learned configuration. This final model is expected to yield improved generalization performance and serve as a strong baseline for comparison against the first architecture (`CNN_LSTM`).

In [9]:
from src.models.model_2 import TCN_STFT_Classifier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TCN_STFT_Classifier(
    num_classes=4,
    hop_length = 128,
    n_fft = 256,
    kernel_size = 3, 
    hidden_channels=  [64, 128, 128],
    dropout = 0.2,
    num_levels = 3,
    device=device,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = torch.nn.CrossEntropyLoss()

trainer = Trainer(model, optimizer, criterion, augment_data=False, device=device)

history = trainer.fit(train_loader, val_loader, epochs=50)

train_loss, train_f1 = trainer.evaluate(train_loader)

val_loss, val_f1 = trainer.evaluate(val_loader)

print(train_f1, val_f1)

c:\Users\toby_\AMLS\venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/50 - Train Loss: 0.9455 - Train F1: 0.2654 - Val Loss: 0.7854 - Val F1: 0.3621
Epoch 2/50 - Train Loss: 0.7761 - Train F1: 0.4506 - Val Loss: 0.7217 - Val F1: 0.5176
Epoch 3/50 - Train Loss: 0.7156 - Train F1: 0.4853 - Val Loss: 0.6634 - Val F1: 0.5686
Epoch 4/50 - Train Loss: 0.6795 - Train F1: 0.5269 - Val Loss: 0.6553 - Val F1: 0.5142
Epoch 5/50 - Train Loss: 0.6405 - Train F1: 0.5714 - Val Loss: 0.6434 - Val F1: 0.4684
Epoch 6/50 - Train Loss: 0.6368 - Train F1: 0.5711 - Val Loss: 0.6879 - Val F1: 0.5538
Epoch 7/50 - Train Loss: 0.6140 - Train F1: 0.5873 - Val Loss: 0.6863 - Val F1: 0.4769
Epoch 8/50 - Train Loss: 0.5865 - Train F1: 0.6157 - Val Loss: 0.6351 - Val F1: 0.4949
Epoch 9/50 - Train Loss: 0.5819 - Train F1: 0.6471 - Val Loss: 0.5858 - Val F1: 0.6015
Epoch 10/50 - Train Loss: 0.5536 - Train F1: 0.6607 - Val Loss: 0.5457 - Val F1: 0.6634
Epoch 11/50 - Train Loss: 0.5548 - Train F1: 0.6483 - Val Loss: 0.5744 - Val F1: 0.5991
Epoch 12/50 - Train Loss: 0.5391 - Train 

## Lets use augment data for improving performance of the model training. First with ECG architechture. 

In [7]:
# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_grid = {
    "lst_hidden_size": [32, 64, 128],
    
    "learning_rate": [.01, 0.001, 0.0005],
    
    "dropout": [0.1, 0.2, .5],
}
fixed = {
    "num_classes": 4,
    "signal_length": X_train.shape[1],
    "n_fft": 512,
    "hop_length": 256,
    "conv1_padding": 1,
    "conv2_padding": 1,
    "conv1_kernel": 3,
    "conv2_kernel": 3,
    "lstm_num_layers": 1,
    "conv1_channels": 32,
    "conv2_channels": 32
}

results = hyperparameter_search(
    ECGNet,
    param_grid,
    fixed,
    device=device,
    epochs=7,
    train_loader=train_loader,
    val_loader=val_loader,
    augmented_data = True
)


🔧 Training with config: {'num_classes': 4, 'signal_length': 18286, 'n_fft': 512, 'hop_length': 256, 'conv1_padding': 1, 'conv2_padding': 1, 'conv1_kernel': 3, 'conv2_kernel': 3, 'lstm_num_layers': 1, 'conv1_channels': 32, 'conv2_channels': 32, 'lst_hidden_size': 32, 'learning_rate': 0.01, 'dropout': 0.1}
Epoch 1/7 | Train Loss: 1.0292 | Train F1: 0.1876 | Val Loss: 1.0027 | Val F1: 0.1856
Epoch 2/7 | Train Loss: 1.0173 | Train F1: 0.1856 | Val Loss: 1.0097 | Val F1: 0.1856
Epoch 3/7 | Train Loss: 1.0164 | Train F1: 0.1853 | Val Loss: 1.0110 | Val F1: 0.1856
Epoch 4/7 | Train Loss: 1.0150 | Train F1: 0.1856 | Val Loss: 1.0059 | Val F1: 0.1856
Epoch 5/7 | Train Loss: 1.0131 | Train F1: 0.1853 | Val Loss: 1.0083 | Val F1: 0.1856
Epoch 6/7 | Train Loss: 1.0150 | Train F1: 0.1853 | Val Loss: 1.0042 | Val F1: 0.1856
Epoch 7/7 | Train Loss: 1.0169 | Train F1: 0.1853 | Val Loss: 1.0052 | Val F1: 0.1856

🔧 Training with config: {'num_classes': 4, 'signal_length': 18286, 'n_fft': 512, 'hop_leng

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ECGNet(
    num_classes=4,
    n_fft=512,
    hop_length=256,
    conv1_padding=1,
    conv2_padding=1,
    conv1_kernel=3,
    conv2_kernel=3,
    lstm_num_layers=1,
    conv1_channels=32,
    conv2_channels=32,
    lst_hidden_size=128,
    dropout=0.1,
    signal_length=X_train.shape[1],
    device=device,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

criterion = torch.nn.CrossEntropyLoss()

trainer = Trainer(model, optimizer, criterion, augment_data=True, device=device)

history = trainer.fit(train_loader, val_loader, epochs=50)

train_loss, train_f1 = trainer.evaluate(train_loader)

val_loss, val_f1 = trainer.evaluate(val_loader)


cm, report = trainer.detailed_metrics(val_loader, class_names=["class_0", "class_1", "class_2", "class_3"])
print("📊 Confusion Matrix:\n", cm)


Epoch 1/50 - Train Loss: 0.9813 - Train F1: 0.1947 - Val Loss: 0.9244 - Val F1: 0.2064
Epoch 2/50 - Train Loss: 0.9182 - Train F1: 0.2427 - Val Loss: 0.8393 - Val F1: 0.2887
Epoch 3/50 - Train Loss: 0.8469 - Train F1: 0.3063 - Val Loss: 0.7759 - Val F1: 0.3293
Epoch 4/50 - Train Loss: 0.7960 - Train F1: 0.3824 - Val Loss: 0.7351 - Val F1: 0.5127
Epoch 5/50 - Train Loss: 0.7635 - Train F1: 0.4649 - Val Loss: 0.6777 - Val F1: 0.5525
Epoch 6/50 - Train Loss: 0.7269 - Train F1: 0.5303 - Val Loss: 0.6462 - Val F1: 0.6219
Epoch 7/50 - Train Loss: 0.6914 - Train F1: 0.5627 - Val Loss: 0.6242 - Val F1: 0.6131
Epoch 8/50 - Train Loss: 0.6543 - Train F1: 0.6150 - Val Loss: 0.6066 - Val F1: 0.5860
Epoch 9/50 - Train Loss: 0.6303 - Train F1: 0.6278 - Val Loss: 0.5689 - Val F1: 0.6845
Epoch 10/50 - Train Loss: 0.6258 - Train F1: 0.6345 - Val Loss: 0.5930 - Val F1: 0.6345
Epoch 11/50 - Train Loss: 0.6069 - Train F1: 0.6465 - Val Loss: 0.5706 - Val F1: 0.6736
Epoch 12/50 - Train Loss: 0.5895 - Train 

In [ ]:
cm, report = trainer.detailed_metrics(val_loader, class_names=["class_0", "class_1", "class_2", "class_3"])
print("\n📄 Classification Report:\n", report)


📄 Classification Report:
               precision    recall  f1-score   support

     class_0     0.8238    0.9532    0.8838       363
     class_1     0.6333    0.7037    0.6667        54
     class_2     0.8067    0.5455    0.6508       176
     class_3     0.8125    0.5909    0.6842        22

    accuracy                         0.8016       615
   macro avg     0.7691    0.6983    0.7214       615
weighted avg     0.8018    0.8016    0.7909       615



In [14]:
from src.models.model_2 import TCN_STFT_Classifier


param_grid = {
    # Configuraciones donde len(hidden_channels) == num_levels
    
        'hidden_channels': [[64,128,128,128],[128,128,128,128]],
    'dropout': [0.1, 0.2, 0.3],
    'kernel_size': [3, 5],
    'num_levels': [3,4]
}


fixed = {
    "num_classes": 4,
    "n_fft": 256,
    "hop_length": 128,
    "kernel_size": 3,
    "learning_rate" : .001,
}

results = hyperparameter_search(
    TCN_STFT_Classifier,
    param_grid,
    fixed,
    device=device,
    epochs=5,
    train_loader=train_loader,
    val_loader=val_loader,
    augmented_data = True
)


🔧 Training with config: {'num_classes': 4, 'n_fft': 256, 'hop_length': 128, 'kernel_size': 3, 'learning_rate': 0.001, 'hidden_channels': [64, 128, 128, 128], 'dropout': 0.1, 'num_levels': 3}
Epoch 1/5 | Train Loss: 0.9341 | Train F1: 0.2667 | Val Loss: 0.8246 | Val F1: 0.3874
Epoch 2/5 | Train Loss: 0.7934 | Train F1: 0.3967 | Val Loss: 0.7065 | Val F1: 0.4543
Epoch 3/5 | Train Loss: 0.7096 | Train F1: 0.4864 | Val Loss: 0.6723 | Val F1: 0.5496
Epoch 4/5 | Train Loss: 0.6553 | Train F1: 0.5339 | Val Loss: 0.6016 | Val F1: 0.6278
Epoch 5/5 | Train Loss: 0.6234 | Train F1: 0.5856 | Val Loss: 0.6080 | Val F1: 0.6162

🔧 Training with config: {'num_classes': 4, 'n_fft': 256, 'hop_length': 128, 'kernel_size': 3, 'learning_rate': 0.001, 'hidden_channels': [64, 128, 128, 128], 'dropout': 0.1, 'num_levels': 4}
Epoch 1/5 | Train Loss: 0.9239 | Train F1: 0.2720 | Val Loss: 0.8040 | Val F1: 0.3220
Epoch 2/5 | Train Loss: 0.7996 | Train F1: 0.4021 | Val Loss: 0.7239 | Val F1: 0.4130
Epoch 3/5 | Tr

In [ ]:
from src.models.model_2 import TCN_STFT_Classifier
from src.models.model_trainer import Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = TCN_STFT_Classifier(
    num_classes=4,
    hop_length = 128,
    n_fft = 256,
    kernel_size = 5, 
    hidden_channels=  [128, 128, 128, 128],
    dropout = 0.1,
    num_levels = 3,
    device=device,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = torch.nn.CrossEntropyLoss()

trainer = Trainer(model, optimizer, criterion, device=device, augment_data = True)

history = trainer.fit(train_loader, val_loader, epochs=50)

train_loss, train_f1 = trainer.evaluate(train_loader)

val_loss, val_f1 = trainer.evaluate(val_loader)

print(train_f1, val_f1)

cm, report = trainer.detailed_metrics(val_loader, class_names=["class_0", "class_1", "class_2", "class_3"])
print("\n📄 Classification Report:\n", report)


c:\Users\toby_\AMLS\venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/50 - Train Loss: 0.9240 - Train F1: 0.3045 - Val Loss: 0.8471 - Val F1: 0.3358
Epoch 2/50 - Train Loss: 0.7996 - Train F1: 0.4088 - Val Loss: 0.7389 - Val F1: 0.3822
Epoch 3/50 - Train Loss: 0.7313 - Train F1: 0.4693 - Val Loss: 0.6555 - Val F1: 0.5936
Epoch 4/50 - Train Loss: 0.6751 - Train F1: 0.5524 - Val Loss: 0.6319 - Val F1: 0.6317
Epoch 5/50 - Train Loss: 0.6157 - Train F1: 0.5948 - Val Loss: 0.6183 - Val F1: 0.6315
Epoch 6/50 - Train Loss: 0.5908 - Train F1: 0.6158 - Val Loss: 0.5669 - Val F1: 0.6596
Epoch 7/50 - Train Loss: 0.5812 - Train F1: 0.6365 - Val Loss: 0.5825 - Val F1: 0.6478
Epoch 8/50 - Train Loss: 0.5544 - Train F1: 0.6596 - Val Loss: 0.5485 - Val F1: 0.6885
Epoch 9/50 - Train Loss: 0.5540 - Train F1: 0.6548 - Val Loss: 0.5741 - Val F1: 0.6793
Epoch 10/50 - Train Loss: 0.5542 - Train F1: 0.6469 - Val Loss: 0.5419 - Val F1: 0.6842
Epoch 11/50 - Train Loss: 0.5400 - Train F1: 0.6639 - Val Loss: 0.5479 - Val F1: 0.6924
Epoch 12/50 - Train Loss: 0.5264 - Train 